Natural Language Processing Assignment1 1951882<br>
Python Code & Report

In [ ]:
import json
import re
import nltk
import math
import random
from nltk import *
from nltk.corpus import wordnet

with open("signal-news1.jsonl", 'r') as f:
    # Read .json file and Lowercase the text
    text = [json.loads(line.lower()[:]) for line in f]

with open("positive-words.txt") as f:
    positive_words = f.read()

with open("negative-words.txt") as f:
    negative_words = f.read()


Part A. Text Preprocessing Q1 a) - d)<br><br>
Firstly, all the texts are reduced to lowercase directly when opening the json file. Since there are four steps to preprocess the texts, each step will return a processed text. Therefore, to avoid writing a similar code after each process, I define a function called **<font>parse_and_clean</font>**. It will return a list which stores the preprocessed strings. In this function, the matched patterns are substituted by space and then ‘None’ value in the strings are filtered. <br><br>
To clean the texts, four patterns are defined to be used in matching target strings. Before removing the non-alphanumeric characters, URLs are removed firstly in order not to break the URLs’ format. After this, the remaining steps are taken. And the final text is stored in a list text_remove4. 


In [ ]:
# Define a function to parse and clean the texts by using regular expressions
def parse_and_clean(pattern, signal_news1):
    text_remove = list(filter(None, [re.sub(pattern, r'', str(line)) for line in signal_news1]))
    return text_remove


# Step1. Remove the URL
pattern1 = re.compile(r'http[s]?://(?:[a-zA-Z0-9]|[$-_@.&+]|(?:%[0-9a-zA-Z][0-9a-zA-Z]))+')
text_remove1 = parse_and_clean(pattern1, text)

# Step2 Remove non-alphanumeric characters and spaces by regular expression ([^\sa-zA-Z0-9])
pattern2 = re.compile(r'[^\sa-zA-Z0-9]')
text_remove2 = parse_and_clean(pattern2, text_remove1)

# Step3 Remove words with only one character
pattern3 = re.compile(r'(\b[a-zA-Z]\b)')
text_remove3 = parse_and_clean(pattern3, text_remove2)

# Step4 Remove numbers that are fully made of digits
pattern4 = re.compile(r'(\b\d+\b)')
text_remove4 = parse_and_clean(pattern4, text_remove3)

Part A. Text Preprocessing Q2<br><br>
To reduce the vocabulary size, texts need to be lemmatized. Before doing so, a POS tagger is used to for classification. To identify the parts-of-speech of a given word, **<font>word_tokenize()</font>** function is used to split the words in each sub-list of **<font>text_removed4</font>** into tokens. Then, **<font>nltk.pos_tag</font>** function is taken. Results give a set of tuples with two elements: first one is token and the second one is the corresponding tagger. Therefore, a list called **<font>tagged</font>** is used to store these tuples. Since the tagger is pretrained from Penn Treebank project, a **<font>get_wordnet_pos</font>** function is defined to map the treebank tags to WordNet POS names. It is accomplished by identifying the first letter of the tag. As a result, lemmas are stored in a list called **<font>lemma</font>** and taggers are removed after POS tagging. After this normalization, the final **<font>tagged</font>** consists of sub-lists, each sub-list represents a news story and words in the news story are lemmatized. 


In [ ]:
# Use pos_tagger to tag word in news
tagged = []

for items in text_remove4:
    tagged.append(pos_tag(word_tokenize(items)))


def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None


wnl = WordNetLemmatizer()
lemma = []
for lists in tagged:
    for n in range(len(lists)):
        wordnet_pos = get_wordnet_pos(lists[n][1]) or wordnet.NOUN
        lists[n] = list(lists[n])  # Change the tuple into list in order to make changes
        lists[n][0] = wnl.lemmatize(lists[n][0], pos=wordnet_pos)  # Lemmatize the words according to POS tagging
        lemma.append(wnl.lemmatize(lists[n][0], pos=wordnet_pos))  # Append the lemmatized words into a lemma list
        lists[n].pop()  # Remove the tag
        lists[n] = ''.join(lists[n])  # Convert the type of words from list to strings

print("Here are the lemmatized words: ", lemma)


Part B. N-grams Q1 & Q2<br><br>
According to the lemmas that obtained in the above, the **<font>len()</font>** is used to calculate the number of tokens in **<font>lemma</font>**. Then, **<font>set()</font>** is used to avoid duplicated words in **<font>lemma</font>** and its length is the size of vocabulary. For the top 25 trigrams of the corpus, words are stored in trigram type by using **<font>trigrams()</font>**. According to the frequency distribution, the 25 most common trigrams are obtained by using **<font>most_common(25)</font>** function.

In [ ]:
# Q1. Number of tokens and vocabulary size

N = len(lemma)
V = len(set(lemma))

print("Number of tokens (N) is ", N)
print("Vocabulary size (V) is ", V)


# Q2. List trigrams and sort it by number of occurrences
trigram = nltk.trigrams(lemma)
tri_freqdist = nltk.FreqDist(trigram)
print("Here is the list of top 25 trigrams: \n", tri_freqdist.most_common(25))

Part B. N-grams Q3 <br><br>
To compute the number of positive and negative word counts in the corpus, words in **<font>lemma</font>** are restored in the type of unigram. According to the frequency distribution of this unigram by using **<font>nltk.freqdist</font>**, each token and its corresponding frequency is obtained. Therefore, this problem can be solved by matching the keys of **<font>nltk.freqdist</font>** with the positive words and negative words separately. Each time when the match successes, the number of positive word (**<font>num_positive</font>**) or negative word (**<font>num_negative</font>**) increases by one. After traversing all the tokens in the corpus, computation is done. 


In [ ]:
# Create a frequency distribution of an uni-gram
uni_freqdist = nltk.FreqDist(lemma)

positive_words_list = positive_words.splitlines()
negative_words_list = negative_words.splitlines()

# Count
num_positive = 0
num_negative = 0

for k, v in uni_freqdist.items():
    if k in positive_words_list[:]:
        num_positive += 1

    if k in negative_words_list[:]:
        num_negative += 1

print("Number of positive word counts in the corpus: ", num_positive)
print("Number of negative word counts in the corpus: ", num_negative)

Part B. N-grams Q4<br><br>
For the question4, **<font>tagged</font>** is reconsidered to be used because we focus on the news story entirely, rather than the single words in it. Here, a **<font>count_pos_neg</font>** function is defined to calculate the number of positive words (**<font>pos</font>**) as well as number of negative words (**<font>neg</font>**) that occurred in each story. Therefore, each story has a list **<font>[pos, neg]</font>** which contains two values, **<font>pos</font>** and **<font>neg</font>**. Then, a simple comparison between them is done as follow. Each time when the **<font>pos</font>** is greater than **<font>neg</font>**, the number of news with more positive words (**<font>num_more_positive_news</font>**) is increased by one. Otherwise, the number of news with more negative words (**<font>num_more_negative_news</font>**) increases. 


In [ ]:
# Define a function to count the positive words and negative words in each sublist
def count_pos_neg(item):
    i = 0
    pos = 0
    neg = 0
    while i < len(item):
        if item[i] in positive_words_list[:]:
            pos += 1

        if item[i] in negative_words_list[:]:
            neg += 1

        i += 1

    return [pos, neg]


num_more_positive_news = 0
num_more_negative_news = 0

# Compare the number of positive words and negative words
for lists in tagged:
    count = count_pos_neg(lists)

    if count[0] > count[1]:
        num_more_positive_news += 1

    if count[0] < count[1]:
        num_more_negative_news += 1

print("Number of positive news stories: ", num_more_positive_news)
print("Number of negative news stories: ", num_more_negative_news)

Part C. Language Models Q1<br><br>
To compute the language model for trigrams, the corpus **<font>tagged</font>** is separated into two sets (train set: **<font>train_tagged</font>** and test set: **<font>test_tagged</font>**). Here, **<font>train_tagged</font>** contains the first 16,000 rows of the corpus, that is the first 16,000 items in the **<font>tagged</font>** list.  Then, rest of items in the **<font>tagged</font>** are testing data for the model. To begin with, a placeholder for model is built by using default dictionary because it can automatically create an entry for the new key, corresponding to a default value. In this model, lists in **<font>train_tagged</font>** are split into trigrams with the help of NLTK. Then, by defining w1, w2, w3 in each trigram, frequency of each combination (w1_w2 and w1_w2_w3) occurred in it are calculated. The numerator is the total counts of w3 together with w1 and w2 in **<font>train_tagged</font>**, while the denominator is the occurrence of the two proceeding words of w3 (w1 and w2 only) in **<font>train_tagged</font>**.<br><br>
However, there are too many zeros in calculation because of the sparsity of data. Add-one estimation of probabilities is taken. That is, for each occurrence of w1_w2_w3, frequency adds 1. For each occurrence of the proceeding words (w1_w2), frequency adds V (**<font>len(add_v_train)</font>**), which is the total number of unique bigrams in the training corpus. V is calculated by splitting **<font>tagged</font>** into bigrams, and the total number of keys in the bigram frequency distribution is the value. Further, since the denominator will be much greater than the numerator, the probabilities may end up with a floating-point underflow. Thus, log space is taken instead. <br><br>
After obtaining the language model, a sentence which starts with “ be this ” is generated instead of “is this” because ‘is’ is lemmatized as ‘be’ in the previous steps. The effect of this difference is slight because ‘this’ can only followed by ‘is’ in common sense. As a result, the sentence is generated word by word. By sorting the dictionary which contains all possible next words in a descending order, the first one will be appended into the text. Then, same method is applied for the next possible words. Once the length of the text reaches 10, the loop will stop. 


In [ ]:
# Q1. Compute language models for trigrams and

train_tagged = tagged[:16000]
test_tagged = tagged[16000:]

# Create a placeholder for model
model = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurrence
for lists in train_tagged:
    for w1, w2, w3 in trigrams(lists, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1


# Number of the unique bi-grams (V), which will be added in the denominator
add_v_train = []
for lists in train_tagged:
    bigram_train = FreqDist(bigrams(lists))
    for k, v in bigram_train.items():
        if k not in add_v_train:
            add_v_train.append(k)
print("Total number of unique bigrams:", len(add_v_train))


# Transforms the counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        numerator = model[w1_w2][w3] + 1
        denominator = total_count + len(add_v_train)
        model[w1_w2][w3] *= int(math.log(numerator) - math.log(denominator))  # Transform the probability into log

# Produce a sentence of 10 words by appending the most likely next word each time
text = ["be", "this"]
EOS = False  # End-of-Sentence

while not EOS:
    dic = sorted(dict(model[tuple(text[-1:])]).items(), key=lambda d: d[1], reverse=True)
    text.append(dic[0])
    if len(text) == 10:
        EOS = True
print(' '.join([t for t in text if t]))

PartC. Language Models Q2<br><br>
According to the language model obtained in Q1, perplexity of this model is calculated based on the testing dataset (**<font>test_tagged</font>**). To calculate it, numerator and denominator are obtained in the same procedure as above. Since add-1 smoothing has been adopted on the model, probability of each list in the **<font>test_tagged</font>** can be obtained directly. Here, pi represents the sum of each probability in the LM.  And the perplexity is calculated based on the equation given in the lecture slides.


In [ ]:
# Q2. Compute the perplexity
pi = 0
perplexity = 0

for lists in test_tagged:
    for w1, w2, w3 in trigrams(lists, pad_right=True, pad_left=True):
        for w1_w2 in model:
            for w3 in model[w1_w2]:
                pi += model[w1_w2][w3]
                perplexity += ((1 / pi) ** len(lists))

print(perplexity)
